<a href="https://colab.research.google.com/github/dadossowhat/bergerson/blob/main/INJE%C3%87%C3%83O_CRIATIVOS_BIGBEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:

from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.service_account import Credentials
import gspread
from google.auth.transport.requests import Request
import requests
# Import the google.auth module
import google.auth
import pandas as pd # Import the pandas module

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly', 'https://www.googleapis.com/auth/drive.readonly']
creds, _ = google.auth.default(scopes=SCOPES)
service = build('drive', 'v3', credentials=creds) # Change from 'sheets' to 'drive'

# Definir o ID da pasta do Google Drive
folder_id = '1xFN4q5KPjDUfDgJS9pTUtOQ_Dzam0_r4'

# Função para listar arquivos .png de uma pasta e gerar o DataFrame com link_criativo e ad_name
def list_png_files_in_folder(folder_id):
    query = f"'{folder_id}' in parents and mimeType='image/png'"
    results = service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])

    if not files:
        print('Nenhum arquivo PNG encontrado.')
        return pd.DataFrame()

    # Criar uma lista de dicionários com ad_name e link_criativo
    file_data = []
    for file in files:
        file_name = file['name'].split('.png')[0]  # Remove a extensão .png
        file_link = f"https://drive.google.com/uc?id={file['id']}"
        file_data.append({'ad_name': file_name, 'thumbnail_url': file_link})

    # Criar DataFrame com os dados
    df = pd.DataFrame(file_data)
    return df

# Listar os arquivos e mostrar o DataFrame
df_png_files = list_png_files_in_folder(folder_id)

df_unique_ad_names = df_png_files.drop_duplicates(subset='ad_name').reset_index(drop=True)

# Exibir o DataFrame filtrado
print(df_unique_ad_names)



                                              ad_name  \
0   sowhat_bigben_aon_bb032_pr_awareness_alcance_m...   
1   sowhat_bigben_aon_bb032_pr_awareness_alcance_m...   
2   sowhat_bigben_aon_bb032_pr_awareness_alcance_m...   
3   sowhat_bigben_aon_bb032_pr_awareness_alcance_m...   
4   sowhat_bigben_aon_bb032_pr_awareness_alcance_m...   
5   sowhat_bigben_aon_bb032_pr_awareness_alcance_m...   
6   sowhat_bigben_aon_bb032_pr_awareness_alcance_m...   
7   sowhat_bigben_aon_bb033_pr-sc-sp_consideracao_...   
8   sowhat_bigben_aon_bb033_pr-sc-sp_consideracao_...   
9   sowhat_bigben_aon_bb033_pr-sc-sp_consideracao_...   
10  sowhat_bigben_aon_bb033_pr-sc-sp_consideracao_...   
11  sowhat_bigben_aon_bb033_pr-sc-sp_consideracao_...   
12  sowhat_bigben_aon_bb033_pr-sc-sp_consideracao_...   
13  sowhat_bigben_aon_bb033_pracas-prioritarias_co...   
14  sowhat_bigben_aon_bb033_pr-sc-sp_consideracao_...   
15  sowhat_bigben_aon_bb033_pr-sc-sp_consideracao_...   
16  sowhat_bigben_aon_bb033_pr-

In [12]:
# prompt: faça uma função que grave o dataframe na planilha de id '1-TepqRlckNN7QevKBxHt7FdW3xz7d0eFIc1e-0fD178' logo após limpar a planilha

def write_dataframe_to_sheet(dataframe, spreadsheet_id):
  """
  Limpa a planilha e escreve o DataFrame na primeira aba.

  Args:
      dataframe: DataFrame a ser escrito na planilha.
      spreadsheet_id: ID da planilha do Google Sheets.
  """

  try:
    gc = gspread.authorize(creds)
    sheet = gc.open_by_key(spreadsheet_id).sheet1

    # Limpar a planilha (exceto a primeira linha)
    sheet.clear()

    # Escrever o DataFrame na planilha
    sheet.update([dataframe.columns.values.tolist()] + dataframe.values.tolist())

    print(f"DataFrame escrito com sucesso na planilha com ID: {spreadsheet_id}")

  except HttpError as error:
    print(f"Um erro ocorreu ao escrever na planilha: {error}")


# ID da planilha para escrita
spreadsheet_id = '1LsKfCNQFjJys72IOBG0vY0UAYt0mNtCQv97G4Mgd_f4'

# Chamar a função para escrever o DataFrame
write_dataframe_to_sheet(df_unique_ad_names, spreadsheet_id)


DataFrame escrito com sucesso na planilha com ID: 1LsKfCNQFjJys72IOBG0vY0UAYt0mNtCQv97G4Mgd_f4


In [13]:
from google.colab import auth
from google.auth import default
from google.cloud import bigquery

def insert_thumbnail_url_to_bigquery():
  """Atualiza a URL da miniatura no BigQuery com base na tabela grade_anuncios_raw_data."""

  auth.authenticate_user()  # Autenticação via popup do Colab
  credentials, project = default()  # Obtém as credenciais padrão

  # Define o projeto explicitamente como 'clientes-sowhat'
  client = bigquery.Client(credentials=credentials, project='clientes-sowhat')

  query = """
  UPDATE `clientes-sowhat.bigben.midia_all_data` A
  SET A.thumbnail_url = B.thumbnail_url
  FROM `clientes-sowhat.bigben.grade_anuncios_raw_data` B
  WHERE A.asset_group = B.ad_name OR A.ad_name = B.ad_name
  """


  query_job = client.query(query)  # Executa a query
  query_job.result()  # Espera até a conclusão

  print("URL de miniaturas atualizada com sucesso no BigQuery.")

# Chamar a função
insert_thumbnail_url_to_bigquery()

URL de miniaturas atualizada com sucesso no BigQuery.
